In [116]:
import pandas as pd
import numpy as np 

In [117]:
# Phase - 1 

In [118]:
def sec_dataframe( ) :
    #Extracting urls
    from bs4 import BeautifulSoup
    import urllib
    import re
    html_page = urllib.request.urlopen("https://www.sec.gov/help/foiadocsinvafoiahtm.html")
    soup = BeautifulSoup(html_page,'lxml')
    type = []
    Date = []
    url = []
    for link in soup.findAll('a', attrs={'href': re.compile("files/data/")}) :
        sample_text = link.get('href')
        if link.string[0] == 'E' :
            type.append('exempt')
        else :
            type.append('Non-exempt')
        get_date = sample_text[sample_text.find('/ia')+3:sample_text.find('/ia')+9]
        Date.append("20"+get_date[-2:]+"-"+""+get_date[0:2]+"-"+get_date[2:4])
        url.append("https://www.sec.gov"+sample_text)
        
    #creating a Data frame 
    d = {'File_URL' : pd.Series(url) , 'Date' : pd.Series(Date) ,'type' : pd.Series(type)}
    sec_df = pd.DataFrame(d)
    sec_df = sec_df[['File_URL','Date','type']]
    sec_df['Date'] = pd.to_datetime(sec_df['Date'])
    return sec_df

In [119]:
sec_df = sec_dataframe()
print(sec_df)

                                              File_URL       Date        type
0    https://www.sec.gov/files/data/information-abo... 2017-12-01      exempt
1    https://www.sec.gov/files/data/information-abo... 2017-12-01  Non-exempt
2    https://www.sec.gov/files/data/information-abo... 2017-11-01  Non-exempt
3    https://www.sec.gov/files/data/information-abo... 2017-11-01      exempt
4    https://www.sec.gov/files/data/information-abo... 2017-10-02      exempt
5    https://www.sec.gov/files/data/information-abo... 2017-10-02  Non-exempt
6    https://www.sec.gov/files/data/information-abo... 2017-09-01  Non-exempt
7    https://www.sec.gov/files/data/information-abo... 2017-09-01      exempt
8    https://www.sec.gov/files/data/information-abo... 2017-08-01      exempt
9    https://www.sec.gov/files/data/information-abo... 2017-08-01  Non-exempt
10   https://www.sec.gov/files/data/frequently-requ... 2017-06-30  Non-exempt
11   https://www.sec.gov/files/data/frequently-requ... 2017-06-3

In [120]:
def get_sec_zip_by_peroid ( periods , is_exempt = False , only_most_recent = False ) :
    urlf = []
    
    if only_most_recent == True :
        if is_exempt ==True :
            date = sec_df[sec_df['type'] == 'exempt']['Date'].max()
            urlf.append(sec_df.loc[(sec_df['Date'] == date) & (sec_df['type'] == 'exempt')]['File_URL'].item())
        else :
            date = sec_df[sec_df['type'] == 'Non-exempt']['Date'].max()
            urlf.append(sec_df.loc[(sec_df['Date'] == date) & (sec_df['type'] == 'Non-exempt')]['File_URL'].item())
        print(urlf)
                
    else :
        per = sec_df['Date'].apply(lambda x:x.strftime('%Y-%m'))
        if is_exempt == True :        
            for i in range(len(periods)) :
                urlf.append(sec_df[( per.isin([periods[i]])) & (sec_df['type'] == 'exempt')]['File_URL'].item())
        else :
            for i in range(len(periods)) :
                urlf.append(sec_df[( per.isin([periods[i]])) & (sec_df['type'] == 'Non-exempt')]['File_URL'].item())
        print(urlf)
        
    #from url, downloading zip file and parsing it's contents
    
    from io import BytesIO
    from zipfile import ZipFile
    import urllib.request
    
    df_get_Period = []
    for i in range(len(urlf)) :
        print(type(urlf[i]))
        url_i = urlf[i]
        url = urllib.request.urlopen(url_i)
        print(url,i)
        with ZipFile(BytesIO(url.read())) as zf:
                print(zf)
                contents = zf.open(zf.namelist()[0])
                filename = contents.name.split('.')[-1]
                print(contents.name)
                if filename != 'xlsx' :
                    if filename == 'CSV' or filename == 'csv' :
                        dfcsv = pd.read_csv(contents,encoding = 'cp1252')
                        df_sec_i = dfcsv
                    else :
                        dftxt  = pd.read_csv(contents,delimiter= '|',encoding = 'cp1252')
                        df_sec_i = dftxt
                else :
                    dfexcel  = pd.ExcelFile(contents)
                    print(dfexcel.sheet_names)
                    df_sec_i = dfexcel.parse(dfexcel.sheet_names)
        df_get_Period.append(df_sec_i)
        print(type(df_get_Period[i]))
        zf.close()
        contents.close()
        print("\n-----------------------------------------------------------------------------------\n")
        
       

In [122]:
get_sec_zip_by_peroid(['2006-06','2009-11','2012-01','2009-09'],False,False)

['https://www.sec.gov/files/data/frequently-requested-foia-document-information-about-registered-investment-advisers-and-exempt/ia060506.zip', 'https://www.sec.gov/files/data/frequently-requested-foia-document-information-about-registered-investment-advisers-and-exempt/ia110209.zip', 'https://www.sec.gov/files/data/frequently-requested-foia-document-information-about-registered-investment-advisers-and-exempt/ia010112.zip', 'https://www.sec.gov/files/data/frequently-requested-foia-document-information-about-registered-investment-advisers-and-exempt/ia090109.zip']
<class 'str'>
<http.client.HTTPResponse object at 0x0000022B05357828> 0
<zipfile.ZipFile file=<_io.BytesIO object at 0x0000022B0E11C410> mode='r'>
2199190_10044_00050000_00050000.txt
<class 'pandas.core.frame.DataFrame'>

-----------------------------------------------------------------------------------

<class 'str'>
<http.client.HTTPResponse object at 0x0000022B07035F60> 1
<zipfile.ZipFile file=<_io.BytesIO object at 0x00000

In [124]:
           # phase -    2

In [125]:
def  asset_managers () :


        import urllib.request
        import json 
        import pandas as pd
        import os

        url_i = "https://doppler.finra.org/doppler-lookup/api/v1/search/firms?hl=true&nrows=99000&query=Blackstone&r=2500&wt=json"
        url = urllib.request.urlopen(url_i)
        data = json.loads(url.read().decode('utf-8'))
        print(type(data))
        d = data['results']['BROKER_CHECK_FIRM']['results']
        d2 = data['results']['BROKER_CHECK_FIRM']
        print(len(d))
        d_fields = {}
        d_highlighted = {}
        count = 0
        data_string = {}
        data_string['results'] = []
        filename = 'result.json'
        for i in range(len(d)) :
                d_fields[i]       = d[i]['fields']
                d_highlighted[i]  = d[i]['highlightedFields']  
                if  d_fields[i]['score']  > 0.4  and 'bc_ia_address_details' in d_fields[i] and d_fields[i]['bc_ia_scope'] == 'ACTIVE' :
                        oa = json.loads(d_fields[i]['bc_ia_address_details'])
                        if 'street1' in  oa['officeAddress'] and oa['officeAddress']['street1'] == '345 PARK AVENUE' :       
                                                    data_string['results'].append(d_fields[i])
                else :
                    continue
        print('--------------------In function------------------------------\n')
        
        data_json = json.dumps(data_string)
        data_json = json.loads(data_json)
        print(type(data_json))
        print('------------------------returning--json data ------------------------\n')
        return data_json

In [126]:
import json 
data = asset_managers()
print('---------------------------after function-- loading json data ---------------------\n')
data_json = json.dumps(data)
data_json = json.loads(data_json)
print(len(data_json['results']))

<class 'dict'>
24
--------------------In function------------------------------

<class 'dict'>
------------------------returning--json data ------------------------

---------------------------after function-- loading json data ---------------------

5


In [127]:
# phase - 3
import pandas as pd

bc_firm_name = [ data_json['results'][i]['bc_firm_name'] for i in range(len(data_json['results'])) ]
bc_source_id = [ data_json['results'][i]['bc_source_id'] for i in range(len(data_json['results'])) ]
dicf = {'bc_firm_name': bc_firm_name , 'bc_source_id' : bc_source_id}
df = pd.DataFrame(dicf)
df

,bc_firm_name,bc_source_id
0,BLACKSTONE MEZZANINE ADVISORS LP,109099
1,BLACKSTONE ALTERNATIVE INVESTMENT ADVISORS LLC,165782
2,BLACKSTONE PROPERTY ADVISORS L.P.,226598
3,BLACKSTONE COMMUNICATIONS ADVISORS I LLC,109106
4,BLACKSTONE DEBT ADVISORS L.P.,120934


In [128]:
#showing all urls based on the bc_source_id
from bs4 import BeautifulSoup
import urllib
import re
urls = []
for i in range(len(df['bc_source_id'])) : 
    url_i = 'https://adviserinfo.sec.gov/IAPD/IAPDFirmSummary.aspx?ORG_PK=' + df['bc_source_id'][i] 
    html_page = urllib.request.urlopen(url_i)
    soup = BeautifulSoup(html_page,'lxml')
    for link in soup.findAll('a', attrs={'href': re.compile("IAPD/Content/Common")}) :
        sample_text = link.get('href')
        urls.append('https://adviserinfo.sec.gov'+sample_text)
urls

['https://adviserinfo.sec.gov/IAPD/Content/Common/crd_iapd_Brochure.aspx?BRCHR_VRSN_ID=446639',
 'https://adviserinfo.sec.gov/IAPD/Content/Common/crd_iapd_Brochure.aspx?BRCHR_VRSN_ID=462964',
 'https://adviserinfo.sec.gov/IAPD/Content/Common/crd_iapd_Brochure.aspx?BRCHR_VRSN_ID=464349',
 'https://adviserinfo.sec.gov/IAPD/Content/Common/crd_iapd_Brochure.aspx?BRCHR_VRSN_ID=442610',
 'https://adviserinfo.sec.gov/IAPD/Content/Common/crd_iapd_Brochure.aspx?BRCHR_VRSN_ID=446632']

In [129]:
#from urls downloading pdf and reading its info and its contents

import pdfx
from io import BytesIO
from PyPDF2 import PdfFileWriter, PdfFileReader


information = []
contents = ''
contents_all = []
for i in range(len(urls)) :
        url = urllib.request.urlopen(urls[i])
        writer = PdfFileWriter()
        remoteFile = url.read()
        memoryFile = BytesIO(remoteFile)
        reader = PdfFileReader(memoryFile)
        contents = ''
        for pageNum in range(reader.getNumPages()):
            currentPage = reader.getPage(pageNum)
            contents = contents + currentPage.extractText()
        pdf_info = reader.getDocumentInfo()
        information.append(pdf_info)
        contents_all.append(contents)
        #use of pdfminer package we can parse contents of pdf 
        #pdf = pdfx.PDFx(urls[i])
        #metadata = pdfx.parse(pdf)
        #print(pdf.info)
    
    

In [130]:
#printing information extracted from the pdf
print(information)
Author = []
for i in range(len(information)) :
    if '/Author' in information[i] and information[i]['/Author'] != "" :
        Author.append(information[i]['/Author'])
    else :
        Author.append(None)
Authors = str(Author)
Authors = pd.Series(data = Author[:])
Authors = Authors.astype('str')
df.loc[:,'Authors'] = Authors

[{'/Title': 'Investment Adviser’s Form ADV, Part 2 A and B “Disclosure Brochure” dated October 12, 2010', '/Author': 'Ascendant Template', '/Creator': 'Microsoft® Word 2010', '/CreationDate': "D:20170411151729-04'00'", '/ModDate': "D:20170411151729-04'00'", '/Producer': 'Microsoft® Word 2010'}, {'/Title': '[Click Here and Type Confidential/Draft]', '/Author': 'TBG', '/Creator': 'Microsoft® Word 2010', '/CreationDate': "D:20170912140300-04'00'", '/ModDate': "D:20170912140300-04'00'", '/Producer': 'Microsoft® Word 2010'}, {'/Producer': '', '/CreationDate': "D:20100101000000Z00'00'", '/ModDate': "D:20100101000000Z00'00'", '/Title': '', '/Creator': '', '/Author': '', '/Subject': '', '/Keywords': ''}, {'/CreationDate': "D:20170331110243-04'00'", '/Creator': 'Acrobat PDFMaker 15 for Word', '/ModDate': "D:20170331115258-04'00'", '/Producer': 'Adobe PDF Library 15.0', '/SourceModified': 'D:20170330214054'}, {'/Title': 'Investment Adviser’s Form ADV, Part 2 A and B “Disclosure Brochure” dated O

In [131]:
df

,bc_firm_name,bc_source_id,Authors
0,BLACKSTONE MEZZANINE ADVISORS LP,109099,Ascendant Template
1,BLACKSTONE ALTERNATIVE INVESTMENT ADVISORS LLC,165782,TBG
2,BLACKSTONE PROPERTY ADVISORS L.P.,226598,None
3,BLACKSTONE COMMUNICATIONS ADVISORS I LLC,109106,None
4,BLACKSTONE DEBT ADVISORS L.P.,120934,Ascendant Template


In [132]:
#similarity ratio finding
import difflib
check = 'Blackstone'
CloseRatio = [    difflib.SequenceMatcher(None, check , word).ratio() for word in df['Authors'] ]
CloseRatio = pd.Series(data = CloseRatio[:])
CloseRatio = CloseRatio.astype('float')
df.loc[:,'CloseRatio'] = CloseRatio

In [133]:
df

,bc_firm_name,bc_source_id,Authors,CloseRatio
0,BLACKSTONE MEZZANINE ADVISORS LP,109099,Ascendant Template,0.285714
1,BLACKSTONE ALTERNATIVE INVESTMENT ADVISORS LLC,165782,TBG,0.153846
2,BLACKSTONE PROPERTY ADVISORS L.P.,226598,None,0.428571
3,BLACKSTONE COMMUNICATIONS ADVISORS I LLC,109106,None,0.428571
4,BLACKSTONE DEBT ADVISORS L.P.,120934,Ascendant Template,0.285714
